# LEFT JOIN with Pandas

In [2]:
import pandas as pd

# 데이터 읽기

In [3]:
orders = pd.read_csv('https://raw.githubusercontent.com/dataitgirls3/Data/master/Orders.csv')
orderDetails = pd.read_csv('https://raw.githubusercontent.com/dataitgirls3/Data/master/OrderDetails.csv')
products = pd.read_csv('https://raw.githubusercontent.com/dataitgirls3/Data/master/Products.csv')
customers = pd.read_csv('https://raw.githubusercontent.com/dataitgirls3/Data/master/Customers.csv')
categories = pd.read_csv('https://raw.githubusercontent.com/dataitgirls3/Data/master/Categories.csv')
suppliers = pd.read_csv('https://raw.githubusercontent.com/dataitgirls3/Data/master/Suppliers.csv')
shippers = pd.read_csv('https://raw.githubusercontent.com/dataitgirls3/Data/master/Shippers.csv')
employees = pd.read_csv('https://raw.githubusercontent.com/dataitgirls3/Data/master/Employees.csv')

# 데이터 확인하기

In [4]:
customers.head(2)

,CustomerID,CustomerName,ContactName,Address,City,PostalCode,Country
0,1,Alfreds Futterkiste,Maria Anders,Obere Str. 57,Berlin,12209,Germany
1,2,Ana Trujillo Emparedados y helados,Ana Trujillo,Avda. de la Constitución 2222,México D.F.,5021,Mexico


In [5]:
orders.head(2)

,OrderID,CustomerID,EmployeeID,OrderDate,ShipperID
0,10248,90,5,1996-07-04,3
1,10249,81,6,1996-07-05,1


In [6]:
orderDetails.head(2)

,OrderDetailID,OrderID,ProductID,Quantity
0,1,10248,11,12
1,2,10248,42,10


In [7]:
products.head(2)

,ProductID,ProductName,SupplierID,CategoryID,Unit,Price
0,1,Chais,1,1,10 boxes x 20 bags,18.0
1,2,Chang,1,1,24 - 12 oz bottles,19.0


## 고객 별 주문수를 알려주세요.

In [8]:
'''
SELECT c.CustomerID
     , COUNT(o.OrderID) as num_orders
FROM Customers AS c
     LEFT JOIN Orders AS o ON c.CustomerID = o.CustomerID
GROUP BY c.CustomerID
'''

'\nSELECT c.CustomerID\n     , COUNT(o.OrderID) as num_orders\nFROM Customers AS c\n     LEFT JOIN Orders AS o ON c.CustomerID = o.CustomerID\nGROUP BY c.CustomerID\n'

* 여기에서 왜 LEFT JOIN이 필요한가? -> 주문을 하나도 하지 않은 고객이 있을 수 있기 때문이다. Orders 테이블에는 존재하지 않지만, Customers 테이블에만 있는 고객들이 있기 때문이다.
* 왜 CustomerName이 아니라 CustomerID를 사용해 GROUP BY를 해야하는가? -> 동명이인이 있을 수 있기 때문이다. 이름과 다르게, CustomerID는 한 사람에 하나만 부여되는 ID입니다. 주민등록번호라고 생각해도 좋겠네요!

In [9]:
customers_order = customers.merge(orders, how='left', on='CustomerID')

In [10]:
# 1번 고객은 Orders 테이블에 정보가 없는가보군요. -> 주문을 한 번도 안 함.
customers_order.head(2)

,CustomerID,CustomerName,ContactName,Address,City,PostalCode,Country,OrderID,EmployeeID,OrderDate,ShipperID
0,1,Alfreds Futterkiste,Maria Anders,Obere Str. 57,Berlin,12209,Germany,NaN,NaN,NaN,NaN
1,2,Ana Trujillo Emparedados y helados,Ana Trujillo,Avda. de la Constitución 2222,México D.F.,5021,Mexico,10308.0,7.0,1996-09-18,3.0


### Q1. 1번 고객이 진짜 없는지 확인 해 볼까요?

In [11]:
# Answer
orders[orders['CustomerID'] == 1]

,OrderID,CustomerID,EmployeeID,OrderDate,ShipperID


### Q2. 고객 중 동명이인이 있나요?

In [12]:
# Answer
c = customers['CustomerName'].value_counts()

In [13]:
c[c != 1]

Series([], Name: CustomerName, dtype: int64)

### 다시 문제로 돌아옵시다.

In [14]:
# size()와 count()의 차이
# size() 는 null값도 카운트함 (몇 줄이 있는지 셈)
customers_num_orders_size = customers_order.groupby('CustomerID')['OrderID'].size()
customers_num_orders_count = customers_order.groupby('CustomerID')['OrderID'].count()

In [15]:
customers_num_orders_size

CustomerID
1     1
2     1
3     1
4     2
5     3
     ..
87    7
88    2
89    2
90    1
91    1
Name: OrderID, Length: 91, dtype: int64

In [16]:
customers_num_orders_count

CustomerID
1     0
2     1
3     1
4     2
5     3
     ..
87    7
88    2
89    2
90    1
91    1
Name: OrderID, Length: 91, dtype: int64

In [17]:
customers_num_orders_count.sort_values()

CustomerID
1      0
26     0
32     0
40     0
22     0
      ..
75     6
65     7
87     7
63     7
20    10
Name: OrderID, Length: 91, dtype: int64

### 주문을 하나도 안한 customer는 몇명인가?

In [18]:
len(customers_num_orders_count[customers_num_orders_count == 0])

17

### 주문을 하나도 안한 customer들의 ID는 무엇인가?

In [19]:
customers_num_orders_count[customers_num_orders_count == 0].index

Int64Index([1, 6, 12, 22, 26, 32, 40, 42, 43, 45, 50, 53, 57, 64, 74, 78, 82], dtype='int64', name='CustomerID')

## 1996년 10월 동안 고객 별 주문 수를 계산하세요.

### 두 쿼리 중 어떤 쿼리가 맞는 결과를 출력할까요?

In [20]:
'''
SELECT c.CustomerID
     , COUNT(o.OrderID)
FROM Customers AS c
     LEFT JOIN Orders AS o ON c.CustomerID = o.CustomerID 
                           AND o.OrderDate BETWEEN '1996-10-01' AND '1996-10-31'
GROUP BY c.CustomerID
'''

"\nSELECT c.CustomerID\n     , COUNT(o.OrderID)\nFROM Customers AS c\n     LEFT JOIN Orders AS o ON c.CustomerID = o.CustomerID \n                           AND o.OrderDate BETWEEN '1996-10-01' AND '1996-10-31'\nGROUP BY c.CustomerID\n"

In [21]:
'''
SELECT c.CustomerID
     , COUNT(o.OrderID)
FROM Customers AS c
     LEFT JOIN Orders AS o ON c.CustomerID = o.CustomerID 
WHERE o.OrderDate BETWEEN '1996-10-01' AND '1996-10-31'
GROUP BY c.CustomerID
'''

"\nSELECT c.CustomerID\n     , COUNT(o.OrderID)\nFROM Customers AS c\n     LEFT JOIN Orders AS o ON c.CustomerID = o.CustomerID \nWHERE o.OrderDate BETWEEN '1996-10-01' AND '1996-10-31'\nGROUP BY c.CustomerID\n"

### 두 쿼리의 차이점
1. orders 테이블에서 원하는 데이터만 고른 후 customer 테이블에 left join
2. orders 테이블을 left join 후 원하는 order date의 데이터를 고름 
    -> 10월사이에 주문건수가 0이라고 떠야하는데 left join 후 데이터를 고르면, 아예 0건인 customer들을 고르지 않기 때문에, 주문하지 않은 customer 의 주문건수가 0이라고 뜨지 않고 아예 없어짐 

### 10월 중 주문만 뽑기

In [22]:
orders_october = orders.set_index('OrderDate').loc['1996-10-01':'1996-10-31',:]
orders_october.head(2)

,OrderID,CustomerID,EmployeeID,ShipperID
OrderDate,,,,
1996-10-01,10318,38,8,2
1996-10-02,10319,80,7,3


In [23]:
orders_october = orders_october.reset_index()
orders_october.head(2)

,OrderDate,OrderID,CustomerID,EmployeeID,ShipperID
0,1996-10-01,10318,38,8,2
1,1996-10-02,10319,80,7,3


### Q3. 1996년 9월 주문만 뽑아서 orders_september에 저장해보세요.

In [24]:
# Answer
orders_september = orders.set_index('OrderDate').loc['1996-09-01':'1996-09-30', :]
orders_september = orders_september.reset_index()
orders_september

,OrderDate,OrderID,CustomerID,EmployeeID,ShipperID
0,1996-09-02,10295,85,2,2
1,1996-09-03,10296,46,6,1
2,1996-09-04,10297,7,5,2
3,1996-09-05,10298,37,6,2
4,1996-09-06,10299,67,4,2
5,1996-09-09,10300,49,2,2
6,1996-09-09,10301,86,8,2
7,1996-09-10,10302,76,4,2
8,1996-09-11,10303,30,7,2
9,1996-09-12,10304,80,1,2


### 다시 원래 문제로 돌아와서,

In [25]:
customers_orders_october = customers.merge(orders_october, how='left', on='CustomerID')

In [26]:
customers_num_orders_october = customers_orders_october.groupby('CustomerID')['OrderID'].count()

In [27]:
customers_num_orders_october.sort_values()

CustomerID
1     0
63    0
62    0
61    0
59    0
     ..
25    2
51    2
87    2
38    2
39    2
Name: OrderID, Length: 91, dtype: int64

### Q4. CustomerID 39번 손님은 1996년 10월 중에 2번 주문을 했네요. 확인 해볼까요?

In [28]:
# Answer
customers_orders_october[customers_orders_october['CustomerID'] == 39]

,CustomerID,CustomerName,ContactName,Address,City,PostalCode,Country,OrderDate,OrderID,EmployeeID,ShipperID
41,39,Königlich Essen,Philip Cramer,Maubelstr. 90,Brandenburg,14776,Germany,1996-10-07,10323.0,4.0,1.0
42,39,Königlich Essen,Philip Cramer,Maubelstr. 90,Brandenburg,14776,Germany,1996-10-09,10325.0,1.0,3.0


## 고객 별 주문금액을 알려주세요.

### Step 1. 주문 별 금액 계산

In [29]:
orders_detail = orders.merge(orderDetails, how='inner', on='OrderID').merge(products, how='inner', on='ProductID')[['OrderID', 'CustomerID', 'OrderDate', 'OrderDetailID', 'Quantity', 'Price']]

In [30]:
orders_detail['total_price'] = orders_detail['Quantity'] * orders_detail['Price']

In [31]:
price_by_order = orders_detail.groupby(['OrderID', 'CustomerID', 'OrderDate'])['total_price'].sum().reset_index()

In [32]:
price_by_order.head(5)

,OrderID,CustomerID,OrderDate,total_price
0,10248,90,1996-07-04,566.00
1,10249,81,1996-07-05,2329.25
2,10250,34,1996-07-08,2267.25
3,10251,84,1996-07-08,839.50
4,10252,76,1996-07-09,4662.50


In [33]:
# 계산 결과가 의도대로 나왔는지 테스트
orders_detail[orders_detail['OrderID'] == 10248]['total_price'].sum()

566.0

### Step 2. 고객 데이터와 붙이기

In [34]:
customers_price_by_order = customers.merge(price_by_order, how='left', on='CustomerID')

In [35]:
customers_price_by_order.head(2)

,CustomerID,CustomerName,ContactName,Address,City,PostalCode,Country,OrderID,OrderDate,total_price
0,1,Alfreds Futterkiste,Maria Anders,Obere Str. 57,Berlin,12209,Germany,NaN,NaN,NaN
1,2,Ana Trujillo Emparedados y helados,Ana Trujillo,Avda. de la Constitución 2222,México D.F.,5021,Mexico,10308.0,1996-09-18,111.0


In [36]:
customers_price_by_order['total_price'] = customers_price_by_order['total_price'].fillna(0)

In [37]:
price_by_customers = customers_price_by_order.groupby('CustomerID')['total_price'].sum()

In [38]:
price_by_customers

CustomerID
1        0.00
2      111.00
3      504.00
4     1723.75
5     5406.90
       ...   
87    5872.50
88    3020.00
89    4416.00
90     566.00
91     573.75
Name: total_price, Length: 91, dtype: float64

### Q5. 89번 고객이 총 주문한 금액은 4416불입니다. 결과가 맞는지 확인하세요.

In [41]:
# Answer
customers_price_by_order[customers_price_by_order['CustomerID'] == 89 ]['total_price'].sum()

4416.0

## 1996년 10월 동안 고객 별 주문 금액을 계산하세요. 10월 중 주문을 한 번도 하지 않은 고객 목록을 주세요.

### Step 1. 10월 주문 별 금액 계산

In [42]:
price_by_order_october = price_by_order.set_index('OrderDate').loc['1996-10-01':'1996-10-31',:].reset_index()

In [43]:
customers_price_by_order_october = customers.merge(price_by_order_october, how='left', on='CustomerID')

### Step 2. 고객 데이터와 붙이기

In [52]:
customers_price_by_order_october.sample()

,CustomerID,CustomerName,ContactName,Address,City,PostalCode,Country,OrderDate,OrderID,total_price
61,57,Paris spécialités,Marie Bertrand,"265, boulevard Charonne",Paris,75012,France,NaN,NaN,NaN


In [54]:
customers_price_by_order_october['total_price'] = customers_price_by_order_october['total_price'].fillna(0)

In [57]:
customers_price_by_order_october.sample()

,CustomerID,CustomerName,ContactName,Address,City,PostalCode,Country,OrderDate,OrderID,total_price
37,36,Hungry Coyote Import Store,Yoshi Latimer,City Center Plaza 516 Main St.,Elgin,97827,USA,NaN,NaN,0.0


In [56]:
customers_price_by_order_october.groupby('CustomerID')['total_price'].sum()

CustomerID
1        0.0
2        0.0
3        0.0
4        0.0
5        0.0
       ...  
87    1837.5
88       0.0
89       0.0
90       0.0
91       0.0
Name: total_price, Length: 91, dtype: float64

## 검증

In [89]:
customer_order = customers.merge(orders, how='left', on='CustomerID')

In [91]:
customer_order_detail = customer_order.merge(orderDetails, how='left', on='OrderID')

In [103]:
customer_order_product_detail = customer_order_detail.merge(products, how='left', on='ProductID')

In [104]:
customer87_order = customer_order_product_detail[customer_order_product_detail['CustomerID'] == 87]

In [106]:
customer87_order

,CustomerID,CustomerName,ContactName,Address,City,PostalCode,Country,OrderID,EmployeeID,OrderDate,ShipperID,OrderDetailID,ProductID,Quantity,ProductName,SupplierID,CategoryID,Unit,Price
508,87,Wartian Herkku,Pirkko Koskitalo,Torikatu 38,Oulu,90110,Finland,10266.0,3.0,1996-07-26,3.0,52.0,12.0,12.0,Queso Manchego La Pastora,5.0,4.0,10 - 500 g pkgs.,38.00
509,87,Wartian Herkku,Pirkko Koskitalo,Torikatu 38,Oulu,90110,Finland,10270.0,1.0,1996-08-01,1.0,60.0,36.0,30.0,Inlagd Sill,17.0,8.0,24 - 250 g jars,19.00
510,87,Wartian Herkku,Pirkko Koskitalo,Torikatu 38,Oulu,90110,Finland,10270.0,1.0,1996-08-01,1.0,61.0,43.0,25.0,Ipoh Coffee,20.0,1.0,16 - 500 g tins,46.00
511,87,Wartian Herkku,Pirkko Koskitalo,Torikatu 38,Oulu,90110,Finland,10320.0,5.0,1996-10-03,3.0,191.0,71.0,30.0,Fløtemysost,15.0,4.0,10 - 500 g pkgs.,21.50
512,87,Wartian Herkku,Pirkko Koskitalo,Torikatu 38,Oulu,90110,Finland,10333.0,5.0,1996-10-18,3.0,227.0,14.0,10.0,Tofu,6.0,7.0,40 - 100 g pkgs.,23.25
513,87,Wartian Herkku,Pirkko Koskitalo,Torikatu 38,Oulu,90110,Finland,10333.0,5.0,1996-10-18,3.0,228.0,21.0,10.0,Sir Rodney's Scones,8.0,3.0,24 pkgs. x 4 pieces,10.00
514,87,Wartian Herkku,Pirkko Koskitalo,Torikatu 38,Oulu,90110,Finland,10333.0,5.0,1996-10-18,3.0,229.0,71.0,40.0,Fløtemysost,15.0,4.0,10 - 500 g pkgs.,21.50
515,87,Wartian Herkku,Pirkko Koskitalo,Torikatu 38,Oulu,90110,Finland,10412.0,8.0,1997-01-13,2.0,439.0,14.0,20.0,Tofu,6.0,7.0,40 - 100 g pkgs.,23.25
516,87,Wartian Herkku,Pirkko Koskitalo,Torikatu 38,Oulu,90110,Finland,10416.0,8.0,1997-01-16,3.0,447.0,19.0,20.0,Teatime Chocolate Biscuits,8.0,3.0,10 boxes x 12 pieces,9.20
517,87,Wartian Herkku,Pirkko Koskitalo,Torikatu 38,Oulu,90110,Finland,10416.0,8.0,1997-01-16,3.0,448.0,53.0,10.0,Perth Pasties,24.0,6.0,48 pieces,32.80


In [107]:
customer87_order_october = customer87_order[(customer87_order['OrderDate'] > '1996-10-01') & (customer87_order['OrderDate'] <= '1996-10-31')]

In [108]:
customer87_order_october

,CustomerID,CustomerName,ContactName,Address,City,PostalCode,Country,OrderID,EmployeeID,OrderDate,ShipperID,OrderDetailID,ProductID,Quantity,ProductName,SupplierID,CategoryID,Unit,Price
511,87,Wartian Herkku,Pirkko Koskitalo,Torikatu 38,Oulu,90110,Finland,10320.0,5.0,1996-10-03,3.0,191.0,71.0,30.0,Fløtemysost,15.0,4.0,10 - 500 g pkgs.,21.50
512,87,Wartian Herkku,Pirkko Koskitalo,Torikatu 38,Oulu,90110,Finland,10333.0,5.0,1996-10-18,3.0,227.0,14.0,10.0,Tofu,6.0,7.0,40 - 100 g pkgs.,23.25
513,87,Wartian Herkku,Pirkko Koskitalo,Torikatu 38,Oulu,90110,Finland,10333.0,5.0,1996-10-18,3.0,228.0,21.0,10.0,Sir Rodney's Scones,8.0,3.0,24 pkgs. x 4 pieces,10.00
514,87,Wartian Herkku,Pirkko Koskitalo,Torikatu 38,Oulu,90110,Finland,10333.0,5.0,1996-10-18,3.0,229.0,71.0,40.0,Fløtemysost,15.0,4.0,10 - 500 g pkgs.,21.50


In [109]:
customer87_order_october['total_price'] = customer87_order_october['Quantity'] * customer87_order_october['Price']

c:\python37\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [110]:
customer87_order_october

,CustomerID,CustomerName,ContactName,Address,City,PostalCode,Country,OrderID,EmployeeID,OrderDate,ShipperID,OrderDetailID,ProductID,Quantity,ProductName,SupplierID,CategoryID,Unit,Price,total_price
511,87,Wartian Herkku,Pirkko Koskitalo,Torikatu 38,Oulu,90110,Finland,10320.0,5.0,1996-10-03,3.0,191.0,71.0,30.0,Fløtemysost,15.0,4.0,10 - 500 g pkgs.,21.50,645.0
512,87,Wartian Herkku,Pirkko Koskitalo,Torikatu 38,Oulu,90110,Finland,10333.0,5.0,1996-10-18,3.0,227.0,14.0,10.0,Tofu,6.0,7.0,40 - 100 g pkgs.,23.25,232.5
513,87,Wartian Herkku,Pirkko Koskitalo,Torikatu 38,Oulu,90110,Finland,10333.0,5.0,1996-10-18,3.0,228.0,21.0,10.0,Sir Rodney's Scones,8.0,3.0,24 pkgs. x 4 pieces,10.00,100.0
514,87,Wartian Herkku,Pirkko Koskitalo,Torikatu 38,Oulu,90110,Finland,10333.0,5.0,1996-10-18,3.0,229.0,71.0,40.0,Fløtemysost,15.0,4.0,10 - 500 g pkgs.,21.50,860.0


In [112]:
customer87_order_october['total_price'] = customer87_order_october['total_price'].fillna(0)

c:\python37\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [114]:
customer87_order_october['total_price'].sum()

1837.5